In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d1 import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions

from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.read_wright_weights import save_weights
from utility.read_wright_weights import reload_weights
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

eval_env = MyEnv()
policies_own = {
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    #"red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
    #          {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    #"red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
    #          {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
policies_enem = {
    "red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    "red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
# policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config_own = ppo.DEFAULT_CONFIG.copy()
config_own = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 1200*25,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_own,  "policy_mapping_fn": policy_mapping_fn}
         }
config_enem = ppo.DEFAULT_CONFIG.copy()
config_enem = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 600*5*10,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_enem,  "policy_mapping_fn": policy_mapping_fn}
         }

res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config_own,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

adversary = ppo.PPOTrainer(config=config_enem,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    
    #trainer.restore(checkpoint_path=model_path)
    #save_weights("blue_0",trainer)
    #save_weights("blue_1",trainer)

    #reload_weights(policy_id="red_0",trainer=trainer,set_policy_id="blue_0")
    #reload_weights(policy_id="red_1",trainer=trainer,set_policy_id="blue_1")
    reload_weights(policy_id="blue_0",trainer=trainer,set_policy_id="blue_0")
    reload_weights(policy_id="blue_1",trainer=trainer,set_policy_id="blue_1")
    #save_weights("red_0",trainer)
    #save_weights("red_1",trainer)


models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
hist_dir = os.path.join('./' + PROJECT + '/hist/')
if not os.path.exists(hist_dir):
    os.makedirs(hist_dir)

for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-04-05 23:24:46,284	INFO trainer.py:2141 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-04-05 23:24:46,285	INFO ppo.py:250 -- In multi-agent mode, policies will be optimized sequentially by the multi

2022-04-05 23:24:59,003	INFO trainable.py:130 -- Trainable.setup took 12.722 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-05 23:24:59,005	WARNING util.py:55 -- Install gputil for GPU system monitoring.


2022-04-05 23:25:10,959	INFO trainable.py:130 -- Trainable.setup took 11.908 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-05 23:25:10,960	WARNING util.py:55 -- Install gputil for GPU system monitoring.


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:

eval_env.reset()
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.self_play = False
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')

        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            trainer.restore(checkpoint_path=model_path)
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_1=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            action_dict1 = [0,0]
            rewards = {"blue_0":0,"blue_1":0}
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                action_dict0 = trainer.compute_single_action(obs['blue_0'],
                                                             state=state_0,prev_action=None,prev_reward=None,
                                                             policy_id='blue_0',explore=False)
                action_dict1 = trainer.compute_single_action(obs['blue_1'],
                                                             state=state_1,prev_action=None,prev_reward=None,
                                                             policy_id='blue_1',explore=False)

                state_0 = action_dict0[1]
                state_1 = action_dict1[1]
                obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0[0], 'blue_1': action_dict1[0]})

                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])


                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()

                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue_"+str(i),steps,env_blue_pos_mod,hist_dir)
                    save_hists("red_"+str(i),steps,env_red_pos_mod,hist_dir)
                    save_hists("mrm_"+str(i),steps,env_mrm_pos_mod,hist_dir)
                    save_weights("blue_0",trainer)
                    save_weights("blue_1",trainer)
                    # print(f'all done at {env.steps}')
                    break
                

            
            #if record_mode == 0:
               # video.release()

ray.shutdown()

-------------------------- Scene: 0 --------------------------

----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
284 blue_0 Shoot at red_1 launch distance : 59032.76532933641 True True
296 blue_0 Shoot at red_1 launch distance : 53230.776969163526 True True
331 red_0 Shoot at blue_0
342 red_0 Shoot at blue_0
397 blue_0 Splash :red_1
400 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.8989993881813123 109.68214350686017
blue_1 False False 1200 0.8989993881813123 -66.08809016092135
-------------------------- Scene: 0 --------------------------
115 blue_0 DOWN
258 blue_1 Shoot at red_1 launch distance : 57840.13148451825 True True
269 blue_1 Shoot at red_1 launch distance : 52109.468081910025 True True
306 red_0 Shoot at blue_1
317 red_0 Shoot at blue_1
364 blue_1 Splash :red_1
371 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 371 0

82 blue_1 DOWN
303 blue_0 Shoot at red_0 launch distance : 57729.42840773387 True True
Same tgt shoot
316 blue_0 Shoot at red_1 launch distance : 53970.44611999345 True True
416 blue_0 Splash :red_0
423 blue_0 Splash :red_1
WIN
blue_0 False True 423 29.010988034047443 117.7179363337698
blue_1 False True 423 12.737229202132548 11.433003621106952
-------------------------- Scene: 0 --------------------------
312 blue_1 Shoot at red_0 launch distance : 57848.973214561156 True True
Same tgt shoot
323 blue_1 Shoot at red_1 launch distance : 57108.67353195086 True True
423 blue_1 Splash :red_0
437 blue_1 Splash :red_1
WIN
blue_0 False True 437 12.646336279789171 3.9092722233409525
blue_1 False True 437 28.738335626471095 119.28499450193782
-------------------------- Scene: 0 --------------------------
669 red_0 Shoot at blue_1
680 red_0 Shoot at blue_1
684 red_1 Shoot at blue_1
695 red_1 Shoot at blue_1
736 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 -0.10500208692301186 -88.50

285 blue_0 Shoot at red_0 launch distance : 59197.5254589637 True True
296 blue_0 Shoot at red_0 launch distance : 53933.99442810178 True True
313 red_1 Shoot at blue_0
324 red_1 Shoot at blue_0
331 blue_1 DOWN
385 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 385 0.35879559993505905 88.4712430489288
blue_1 False False 385 0.39900013326839234 0.9332270230090767
-------------------------- Scene: 0 --------------------------
295 blue_0 Shoot at red_1 launch distance : 59909.65408034755 True True
Same tgt shoot
309 blue_0 Shoot at red_0 launch distance : 54763.73375294694 True True
411 blue_0 Splash :red_1
414 blue_0 Splash :red_0
WIN
blue_0 False True 414 29.196013930685616 136.60731186564055
blue_1 False True 414 12.79891094807692 0.10485482463766793
-------------------------- Scene: 0 --------------------------
270 blue_1 Shoot at red_1 launch distance : 56302.576033900434 True True
Same tgt shoot
281 blue_1 Shoot at red_0 launch distance : 59298.60043012082 True True
378 blue_1

2022-04-06 00:27:22,789	WARNING deprecation.py:46 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


agent_timesteps_total: 61792
custom_metrics: {}
date: 2022-04-06_00-27-22
done: false
episode_len_mean: 835.027027027027
episode_media: {}
episode_reward_max: 167.94170859736292
episode_reward_mean: 23.11768149629391
episode_reward_min: -149.68987154881796
episodes_this_iter: 37
episodes_total: 37
experiment_id: c908197bc72d46f4b0eac61bfb99b0c7
hostname: DESKTOP
info:
  learner:
    blue_0:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 5.517724514007568
        entropy_coeff: 0.0
        kl: 0.13516825437545776
        model: {}
        policy_loss: 0.009893443435430527
        total_loss: 381.3448791503906
        vf_explained_var: -0.30137011408805847
        vf_loss: 381.3079528808594
      train: null
    blue_1:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 5.669012546539307
        entropy_coeff: 0.0
        kl: 0.10488949716091156
      

...

285 blue_0 Shoot at red_0 launch distance : 59208.37395746732 True True
296 blue_0 Shoot at red_0 launch distance : 53461.72445856623 True True
326 red_1 Shoot at blue_0
337 red_1 Shoot at blue_0
397 blue_0: Destroyed
399 blue_0 Splash :red_0
Same tgt shoot
Same tgt shoot
554 blue_1 Shoot at red_1 launch distance : 48864.37664274949 True True
648 blue_1 Splash :red_1
WIN
blue_0 False True 648 12.252330003060306 128.62583694498338
blue_1 False True 648 26.056033706764012 99.2097989222222
-------------------------- Scene: 0 --------------------------
279 blue_0 Shoot at red_1 launch distance : 53385.652391521384 True True
Same tgt shoot
290 blue_0 Shoot at red_0 launch distance : 57022.75439998318 True True
Same tgt shoot
317 blue_1 Shoot at red_0 launch distance : 58388.11236280191 True True
379 blue_0 Splash :red_1
398 blue_0 Splash :red_0
WIN
blue_0 False True 517 12.321083172147002 128.77893395177304
blue_1 False True 517 12.320083172147003 61.186325972147266
------------------------

2022-04-06 06:46:26,037	INFO trainable.py:496 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000007\checkpoint-7
2022-04-06 06:46:26,038	INFO trainable.py:503 -- Current state after restoring: {'_iteration': 7, '_timesteps_total': 215363, '_time_total': 25839.885903835297, '_episodes_total': 283}


agent_timesteps_total: 430726
custom_metrics: {}
date: 2022-04-06_06-46-25
done: false
episode_len_mean: 697.37
episode_media: {}
episode_reward_max: 269.43817038434014
episode_reward_mean: 113.54180188647392
episode_reward_min: -100.96828268770302
episodes_this_iter: 46
episodes_total: 283
experiment_id: c908197bc72d46f4b0eac61bfb99b0c7
hostname: DESKTOP
info:
  learner:
    blue_0:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 6.421503067016602
        entropy_coeff: 0.0
        kl: 0.2821464240550995
        model: {}
        policy_loss: 0.08547231554985046
        total_loss: 156.19509887695312
        vf_explained_var: 0.3230697214603424
        vf_loss: 156.05320739746094
      train: null
    blue_1:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 6.7126641273498535
        entropy_coeff: 0.0
        kl: 0.2637454867362976
        model: 

394 blue_0 Splash :red_1
414 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.899000826148885 120.52216497251217
blue_1 False False 1200 0.893000826148885 -11.095847646531528
-------------------------- Scene: 0 --------------------------
139 blue_1 DOWN
284 blue_0 Shoot at red_0 launch distance : 59509.21785156876 True True
Same tgt shoot
298 blue_0 Shoot at red_1 launch distance : 52933.97897590967 True True
325 red_1 Shoot at blue_0
395 blue_0 Splash :red_1
401 blue_0 Splash :red_0
WIN
blue_0 False True 525 12.284714285714285 136.3259467259529
blue_1 False True 525 12.184714285714284 10.981244348847138
-------------------------- Scene: 0 --------------------------
281 blue_0 Shoot at red_1 launch distance : 56474.31499122806 True True
Same tgt shoot
293 blue_0 Shoot at red_0 launch distance : 53608.575407149256 True True
387 blue_0 Splash :red_1
395 blue_0 Splash :red_0
WIN
blue_0 False True 395 29.61426192529519 128.6447210382298
blue_1 False True 395 12.938320924873251 8

Same tgt shoot
341 blue_1 Shoot at red_1 launch distance : 55790.361109533216 True True
455 blue_1 Splash :red_1
458 blue_1 Splash :red_0
WIN
blue_0 False True 458 12.520445006375459 11.115517113794507
blue_1 False True 458 27.861619678864542 137.36623505012557
-------------------------- Scene: 0 --------------------------
138 blue_1 DOWN
270 blue_0 Shoot at red_1 launch distance : 59589.152331026715 True True
281 blue_0 Shoot at red_1 launch distance : 53813.05576431643 True True
326 red_0 Shoot at blue_0
337 red_0 Shoot at blue_0
365 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 365 0.35892113063513514 81.28338012298423
blue_1 False False 365 -0.10099993603153154 -1.3040192751666828
-------------------------- Scene: 0 --------------------------
279 blue_0 Shoot at red_1 launch distance : 57871.927182403204 True True
Same tgt shoot
290 blue_0 Shoot at red_0 launch distance : 54319.72881657357 True True
390 blue_0 Splash :red_1
394 blue_0 Splash :red_0
WIN
blue_0 False True 394 